In [93]:
from pyspark.sql import SparkSession

def create_spark_session() -> SparkSession:
    spark = (
        SparkSession.builder
        .appName("GoldLayer")
        .enableHiveSupport()
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3minio.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .getOrCreate()
    )
    return spark

spark = create_spark_session()


In [94]:
minio_delta_options = {
    "fs.s3a.access.key": "admin",
    "fs.s3a.secret.key": "senhasegura",
    "fs.s3a.endpoint": "http://minio:9000",
    "fs.s3a.path.style.access": "true",
    "fs.s3a.connection.ssl.enabled": "false"
}

In [95]:
df_team = spark.read.parquet("s3a://bucket-silver-zone/team/")
df_team_attributes = spark.read.parquet("s3a://bucket-silver-zone/team_attributes/")
df_match = spark.read.parquet("s3a://bucket-silver-zone/match/")

#df_team.show()
#df_team_attributes.show()
#df_match.show()

## Criação de métricas por temporada:

In [96]:
#Desempenho do time: Média de gols por partida 
from pyspark.sql.functions import avg, count, col
# Calcular média de gols por time e temporada
df_team_performance = df_match.groupBy("ano", "time_casa_api_id").agg(avg("gols_time_casa").alias("media_gols"),avg("gols_time_visitante").alias("media_gols_fora_de_casa"))
df_team_performance.show()

+---------+----------------+------------------+-----------------------+
|      ano|time_casa_api_id|        media_gols|media_gols_fora_de_casa|
+---------+----------------+------------------+-----------------------+
|2015/2016|            8573|2.3333333333333335|                    1.4|
|2015/2016|            8586|1.8421052631578947|     0.7894736842105263|
|2009/2010|            8569|1.5333333333333334|     0.7333333333333333|
|2012/2013|            9850|1.3157894736842106|     1.0526315789473684|
|2013/2014|            9826|0.9473684210526315|     1.2105263157894737|
|2012/2013|            1601|1.1333333333333333|                    1.4|
|2009/2010|            1957|               1.2|                    0.4|
|2009/2010|            8031|1.2666666666666666|                    0.8|
|2010/2011|            8559|1.7894736842105263|      1.263157894736842|
|2013/2014|            6433| 1.588235294117647|     1.5294117647058822|
|2008/2009|            1601|0.8666666666666667|     0.7333333333

In [97]:
#Eficiência ofensiva: Número total de finalizações certas por time
df_offensive_efficiency = df_match.groupBy("ano", "time_casa_api_id").agg(count("shoton").alias("total_shots_on_target"))

# Realizar o join com `df_team` para incluir o nome do time
df_offensive_efficiency = df_offensive_efficiency.join(df_team, df_offensive_efficiency.time_casa_api_id == df_team.team_api_id, "left") \
    .select("ano", "time_casa_api_id", "team_long_name", "total_shots_on_target")

# Exibir resultados
df_offensive_efficiency.show()


+---------+----------------+--------------------+---------------------+
|      ano|time_casa_api_id|      team_long_name|total_shots_on_target|
+---------+----------------+--------------------+---------------------+
|2015/2016|            8573|         KV Oostende|                    0|
|2015/2016|            8586|   Tottenham Hotspur|                   19|
|2009/2010|            8569|       GKS Bełchatów|                    0|
|2012/2013|            9850|        Norwich City|                   19|
|2013/2014|            9826|      Crystal Palace|                   19|
|2012/2013|            1601|        Ruch Chorzów|                    0|
|2009/2010|            1957|Jagiellonia Biały...|                    0|
|2009/2010|            8031|       Polonia Bytom|                    0|
|2010/2011|            8559|    Bolton Wanderers|                   19|
|2013/2014|            6433|     Go Ahead Eagles|                    0|
|2008/2009|            1601|        Ruch Chorzów|               

In [98]:
#Eficiência defensiva: Média de gols sofridos

# Calcular a média de gols sofridos por time e temporada
df_defensive_efficiency = df_match.groupBy("ano", "time_casa_api_id").agg(avg(df_match.gols_time_casa + df_match.gols_time_visitante).alias("avg_goals_conceded"))

df_defensive_efficiency = df_defensive_efficiency.join(df_team, df_defensive_efficiency.time_casa_api_id == df_team.team_api_id, "left") \
    .select("ano", "time_casa_api_id", "team_long_name", "avg_goals_conceded")

# Exibir resultados
df_defensive_efficiency.show()




+---------+----------------+--------------------+------------------+
|      ano|time_casa_api_id|      team_long_name|avg_goals_conceded|
+---------+----------------+--------------------+------------------+
|2015/2016|            8573|         KV Oostende|3.7333333333333334|
|2015/2016|            8586|   Tottenham Hotspur|2.6315789473684212|
|2009/2010|            8569|       GKS Bełchatów|2.2666666666666666|
|2012/2013|            9850|        Norwich City|2.3684210526315788|
|2013/2014|            9826|      Crystal Palace|2.1578947368421053|
|2012/2013|            1601|        Ruch Chorzów| 2.533333333333333|
|2009/2010|            1957|Jagiellonia Biały...|               1.6|
|2009/2010|            8031|       Polonia Bytom| 2.066666666666667|
|2010/2011|            8559|    Bolton Wanderers|3.0526315789473686|
|2013/2014|            6433|     Go Ahead Eagles|3.1176470588235294|
|2008/2009|            1601|        Ruch Chorzów|               1.6|
|2009/2010|            8021|      

In [99]:
# Classificação dos times com base nas finalizações certas e gols sofridos 
df_team_performance = df_match.groupBy("ano", "time_casa_api_id").agg(avg("gols_time_casa").alias("media_gols"), avg("gols_time_visitante").alias("media_gols_fora_de_casa"))

df_offensive_efficiency = df_match.groupBy("ano", "time_casa_api_id").agg(count("shoton").alias("total_shots_on_target"))

df_defensive_efficiency = df_match.groupBy("ano", "time_casa_api_id").agg(avg(df_match.gols_time_casa + df_match.gols_time_visitante).alias("avg_goals_conceded"))

df_team_performance = df_team_performance.alias("tp").join(
    df_offensive_efficiency.alias("oe"), col("tp.time_casa_api_id") == col("oe.time_casa_api_id"), "left"
).join(
    df_defensive_efficiency.alias("de"), col("tp.time_casa_api_id") == col("de.time_casa_api_id"), "left"
).join(
    df_team.alias("t"), col("tp.time_casa_api_id") == col("t.team_api_id"), "left"
).select(
    col("tp.ano").alias("ano"),  # Garantindo que a 'ano' venha de `df_team_performance`
    col("tp.time_casa_api_id"),
    col("t.team_long_name"),
    col("tp.media_gols"),
    col("tp.media_gols_fora_de_casa"),
    col("oe.total_shots_on_target"),
    col("de.avg_goals_conceded")
)

# Resultado final
df_team_performance.show()

+---------+----------------+-----------------+------------------+-----------------------+---------------------+------------------+
|      ano|time_casa_api_id|   team_long_name|        media_gols|media_gols_fora_de_casa|total_shots_on_target|avg_goals_conceded|
+---------+----------------+-----------------+------------------+-----------------------+---------------------+------------------+
|2015/2016|            8573|      KV Oostende|2.3333333333333335|                    1.4|                    0|1.3333333333333333|
|2015/2016|            8573|      KV Oostende|2.3333333333333335|                    1.4|                    0| 3.466666666666667|
|2015/2016|            8573|      KV Oostende|2.3333333333333335|                    1.4|                    0|3.7333333333333334|
|2015/2016|            8573|      KV Oostende|2.3333333333333335|                    1.4|                    0|1.3333333333333333|
|2015/2016|            8573|      KV Oostende|2.3333333333333335|                  

In [100]:
#Gravação no minio
(
    df_team_performance.write
    .format("delta")
    .options(**minio_delta_options)
    .mode("overwrite")
    .save("s3a://bucket-gold-zone/performance/")
)